
##### spacy_text_classification : Exercise
* In this exercise, you are going to classify whether a given text belongs to one of possible classes ['BUSINESS', 'SPORTS', 'CRIME'].

* you are going to use spacy for pre-processing the text, convert text to numbers and apply different classification algorithms.

In [1]:
#import spacy and load the language model downloaded
import spacy
nlp = spacy.load("en_core_web_lg")

#### About Data: News Category Classifier
* Credits: https://www.kaggle.com/code/hengzheng/news-category-classifier-val-acc-0-65

* This data consists of two columns. - Text - Category
* Text are the description about a particular topic.
* Category determine which class the text belongs to.
* we have classes mainly of 'BUSINESS', 'SPORTS', 'CRIME' and comes under Multi-class classification Problem.

In [2]:
#import pandas library
import pandas as pd
#read the dataset "news_dataset.json" provided and load it into dataframe "df"
df = pd.read_json("news_dataset.json")
#print the shape of data
df.shape
#print the top5 rows
df.head(5)

,text,category
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS
3,This Richard Sherman Interception Literally Sh...,SPORTS
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS


In [8]:
#check the distribution of labels 
df.category.value_counts()

category
CRIME       2500
SPORTS      2500
BUSINESS    2500
Name: count, dtype: int64

In [11]:
#Add the new column "label_num" which gives a unique number to each of these labels 
df['label_num'] = df['category'].map({'CRIME':0,'SPORTS':1,'BUSINESS':2})
#check the results with top 5 rows
df.head(5)

,text,category,label_num
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME,0
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME,0
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS,1
3,This Richard Sherman Interception Literally Sh...,SPORTS,1
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS,2


#### Preprocess the text

In [12]:
#use this utility function to preprocess the text
#1. Remove the stop words
#2. Convert to base form using lemmatisation

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [13]:
#create a new column "preprocessed_text" which store the clean form of given text [use apply and lambda
df['preprocessed_text'] = df['text'].apply(lambda text : preprocess(text))

In [14]:
df.head(5)

,text,category,label_num,preprocessed_text
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME,0,Larry Nassar blame victim say victimize newly ...
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME,0,woman Beats Cancer die fall horse
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS,1,vegas taxpayer spend Record $ 750 million New ...
3,This Richard Sherman Interception Literally Sh...,SPORTS,1,Richard Sherman Interception literally shake W...
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS,2,7 thing totally kill Weed Legalization Buzz


#### Get the spacy embeddings for each preprocessed text

In [24]:
#create a new column "vector" that store the vector representation of each pre-processed text

df['vector'] = df['preprocessed_text'].apply(lambda text: nlp(text).vector) 

In [25]:
#print the top 5 rows

df.head()

,text,category,label_num,preprocessed_text,vector
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME,0,Larry Nassar blame victim say victimize newly ...,"[-0.5585511, -0.29323253, -0.9253956, 0.189389..."
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME,0,woman Beats Cancer die fall horse,"[-0.73039824, -0.43196002, -1.2930516, -1.0628..."
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS,1,vegas taxpayer spend Record $ 750 million New ...,"[-1.9413117, 0.121578515, -3.2996283, 1.511650..."
3,This Richard Sherman Interception Literally Sh...,SPORTS,1,Richard Sherman Interception literally shake W...,"[-1.4702771, -0.685319, 0.57398, -0.31135806, ..."
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS,2,7 thing totally kill Weed Legalization Buzz,"[-1.037173, -1.9495698, -1.7179357, 1.2975286,..."


##### Train-Test splitting

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.vector.values,df['label_num'],test_size=0.2,random_state = 98,stratify=df.label_num)

##### Reshape the X_train and X_test so as to fit for models

In [27]:
# import numpy as np
import numpy as np

#reshapes the X_train and X_test using 'stack' function of numpy. Store the result in new variables "X_train_2d" and "X_test_2d"
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)



##### Attempt 1:

* use spacy glove embeddings for text vectorization.

* use Decision Tree as the classifier.

* print the classification report.

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
#1. creating a Decision Tree model object
clf = DecisionTreeClassifier()
#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d,y_train)
#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)
#4. print the classfication report
classification_report(y_pred,y_test)

'              precision    recall  f1-score   support\n\n           0       0.71      0.71      0.71       500\n           1       0.72      0.72      0.72       503\n           2       0.72      0.73      0.73       497\n\n    accuracy                           0.72      1500\n   macro avg       0.72      0.72      0.72      1500\nweighted avg       0.72      0.72      0.72      1500\n'

##### Attempt 2:

* use spacy glove embeddings for text vectorization.
* use MultinomialNB as the classifier after applying the MinMaxscaler.
* print the classification report.

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

#doing scaling because Negative values will not pass into Naive Bayes models
scaler= MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.fit_transform(X_test_2d)
#1. creating a MultinomialNB model object 
clf = MultinomialNB()
#2. fit with all_train_embeddings(scaled) and y_train
clf.fit(scaled_train_embed,y_train)
#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(scaled_test_embed)
#4. print the classfication report
classification_report(y_pred,y_test)

'              precision    recall  f1-score   support\n\n           0       0.85      0.87      0.86       490\n           1       0.80      0.84      0.82       477\n           2       0.84      0.79      0.82       533\n\n    accuracy                           0.83      1500\n   macro avg       0.83      0.83      0.83      1500\nweighted avg       0.83      0.83      0.83      1500\n'

##### Attempt 3:

* use spacy glove embeddings for text vectorization.
* use KNeighborsClassifier as the classifier after applying the MinMaxscaler.
* print the classification report.

In [34]:
# from  sklearn.neighbors import KNeighborsClassifier


# #1. creating a KNN model object
# clf = KNeighborsClassifier(n_neighbors=5,metric='euclidean')
# #2. fit with all_train_embeddings and y_train
# clf.fit(scaled_train_embed,y_train)
# #3. get the predictions for all_test_embeddings and store it in y_pred
# y_pred = clf.predict(scaled_test_embed)
# #4. print the classfication report
# classification_report(y_pred,y_test)

#### Attempt 4:

* use spacy glove embeddings for text vectorization.
* use RandomForestClassifier as the classifier after applying the MinMaxscaler.
* print the classification report.

In [35]:
from sklearn.ensemble import RandomForestClassifier


#1. creating a Random Forest model object
clf = RandomForestClassifier()
#2. fit with all_train_embeddings and y_train
clf.fit(scaled_train_embed,y_train)
#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(scaled_test_embed)
#4. print the classfication report
classification_report(y_pred,y_test)

'              precision    recall  f1-score   support\n\n           0       0.91      0.81      0.86       559\n           1       0.76      0.85      0.81       447\n           2       0.84      0.85      0.84       494\n\n    accuracy                           0.84      1500\n   macro avg       0.84      0.84      0.84      1500\nweighted avg       0.84      0.84      0.84      1500\n'

#### Attempt 5:

* use spacy glove embeddings for text vectorization.
* use GradientBoostingClassifier as the classifier after applying the MinMaxscaler.
* print the classification report.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()
#2. fit with all_train_embeddings and y_train
clf.fit(scaled_train_embed,y_train)
#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(scaled_test_embed)
#4. print the classfication report
classification_report(y_pred,y_test)

#### Print the confusion Matrix with the best model got

In [ ]:
#finally print the confusion matrix for the best model: GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

from matplotlib.pyplot